In [9]:
import geopandas
import pandas as pd

dict_data = {'Score': {0: 9, 1: 4, 2: 12, 3: 12, 4: 2, 5: 4, 6: 2},
 'LATITUDE': {0: 47.19885642,
  1: 47.19932583,
  2: 47.20058986,
  3: 47.20058986,
  4: 47.19847967,
  5: 47.19980364,
  6: 47.19929947},
 'LONGITUDE': {0: 6.51906939,
  1: 6.52133392,
  2: 6.521687,
  3: 6.521687,
  4: 6.51926344,
  5: 6.52126031,
  6: 6.52197033},
 'Name': {0: 'a', 1: 'a', 2: 'a', 3: 'a', 4: 'b', 5: 'b', 6: 'b'}}

df = pd.DataFrame(dict_data)

gdf = geopandas.GeoDataFrame(
    df,
    geometry=geopandas.points_from_xy(df["LONGITUDE"], df["LATITUDE"]))

gdf.crs = "EPSG:4326"

gdf.explore(
    column="Name",
    tiles="CartoDB positron",
    cmap="viridis_r",
    style_kwds={"style_function":lambda x: {"radius":x["properties"]["Score"]}}
)

In [1]:
!pip install plotly

In [2]:
!pip install geopandas
!pip install mapclassify
!pip install contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 55.8 MB/s eta 0:00:00


In [58]:
from os import XATTR_CREATE
import geopandas as gpd
from mapclassify import classify
import pandas as pd
import plotly.express as px

def AnalysisGeoPandas(n):
# geopandas can deal with JSON file directly with its own API
    cntyDat = gpd.read_file('gz_2010_us_050_00_20m.json', driver = 'GeoJSON')
    print(cntyDat.head(3))
    print("\nType of the data is: {}".format(type(cntyDat)))
    print(f"\nThe type of the geometry column is: {type(cntyDat.geometry)}")
    #GeoDataFrame can deliver a map, while Dataframe needs the orginal GeoJSON-formatted dict for plotly map.
    mObj = cntyDat.to_crs(4326).explore()
    cntyDat.to_crs(4326).explore(column ="CENSUSAREA", vmax=1200).save('map01.html')
# Find top n most common county names
    common = mostCommonName(cntyDat,'NAME',n)
    cntyDat[cntyDat['NAME'].isin(common.index)].to_crs(4326).explore(m = mObj,\
      style_kwds ={"style_function":lambda x: {"color":"red" if x["properties"]["NAME"]==common.index[0] \
                             else ("green" if x["properties"]["NAME"]==common.index[1] else "yellow")}}).save('map02.html')
# Derive summary statistics for states, for example, number of counties, counties with max/min area
    df_stat = cntyDat.groupby(by="STATE").agg(amount_county=("COUNTY", "count"),\
                            min_area=("CENSUSAREA", "min"),max_area=("CENSUSAREA", "max"),\
                            total_area=("CENSUSAREA", "sum"),average_area=("CENSUSAREA", "mean"))
    print("The biggest and smallest county and total/average area of counties in each state:\n", df_stat)
# Join fips code to get full names of states (plesae do a real join using merge)
    print("The statistics results with full names of states: \n", stateInfo(df_stat))


def AnalysisPandas(n):
#pandas can deal with JSON file directly with its own API. but the result needs to be converted
    data = pd.read_json('gz_2010_us_050_00_20m.json')
    cntyDat = pd.json_normalize(data['features'], max_level=1)
    print(cntyDat.head(3))
    print("\nType of the data is: {}".format(type(cntyDat)))
# Find top n most common county names
    mostCommonName(cntyDat,'properties.NAME',n)
# Derive summary statistics for states, for example, number of counties, counties with max/min area
    df_stat = cntyDat.groupby(by="properties.STATE").agg(amount_county=("properties.COUNTY", "count"),\
                            min_area=("properties.CENSUSAREA", "min"),max_area=("properties.CENSUSAREA", "max"),\
                            total_area=("properties.CENSUSAREA", "sum"),average_area=("properties.CENSUSAREA", "mean")).rename_axis('STATE')
    print("The biggest and smallest county and total/average area of counties in each state:\n", df_stat)
# Join fips code to get full names of states (plesae do a real join using merge)
    print("The statistics results with full names of states: \n", stateInfo(df_stat))



# main procedure:
AnalysisGeoPandas(3)
AnalysisPandas(3)

           GEO_ID STATE COUNTY      NAME    LSAD  CENSUSAREA  \
0  0500000US01001    01    001   Autauga  County     594.436   
1  0500000US01009    01    009    Blount  County     644.776   
2  0500000US01017    01    017  Chambers  County     596.531   

                                            geometry  
0  POLYGON ((-86.49677 32.34444, -86.71790 32.402...  
1  POLYGON ((-86.57780 33.76532, -86.75914 33.840...  
2  POLYGON ((-85.18413 32.87053, -85.12342 32.772...  

Type of the data is: <class 'geopandas.geodataframe.GeoDataFrame'>

The type of the geometry column is: <class 'geopandas.geoseries.GeoSeries'>
The most common name county names with frequencies are:
NAME
Washington    31
Franklin      26
Jefferson     26
dtype: int64
The biggest and smallest county and total/average area of counties in each state:
        amount_county  min_area    max_area  total_area  average_area
STATE                                                               
01                67   534.991  

In [14]:
def mostCommonName(cntyDat,paraName,n):
    common = cntyDat.groupby(paraName).size().nlargest(n)
    print('The most common name county names with frequencies are:')
    print(common)
    return common

In [54]:
def fipdict():
  import json
  with open('fipsToState.json') as f:
      fipdict= json.load(f)
  with open('stateCodeToFips.json') as g:
      codedict= json.load(g)
  fippd = pd.DataFrame.from_dict(fipdict, orient="index", columns=['st_name'])\
    .rename_axis('STATE')\
    .reset_index()
  codepd = pd.DataFrame.from_dict(codedict, orient="index", columns=['STATE'])\
    .rename_axis('st_code')\
    .reset_index()
  mergedDF = codepd.merge(fippd, left_on='STATE', right_on ='STATE')
  if mergedDF.loc[mergedDF['st_name'].isna()].size !=0 :
       print('The number of state code file and state fip file doesnot match',set(mergedDF.loc[mergedDF['st_name'].isna()].loc['STATE']))
  return mergedDF

def stateInfo(cntyDat):
# check the integrity of fip codes after merge
    fipDF = fipdict()
    cntyDatMerged = cntyDat.merge(fipDF, how='left', left_on='STATE', right_on ='STATE')
    print('The original fip codes does not include',set(cntyDatMerged.loc[cntyDatMerged['st_name'].isna()]['STATE']))

# add the missed states info into the final dictionary
    fipDF = fipDF._append({'st_code':'PR','STATE':'72','st_name':'PUERTO RICO'}, ignore_index = True)
    cntyDatMerged = cntyDat.merge(fipDF, how='left', left_on='STATE', right_on ='STATE')
    print('The updated fip codes does not include',set(cntyDatMerged.loc[cntyDatMerged['st_name'].isna()]['STATE']))
    return cntyDatMerged